In [246]:
import pandas as pd
from cleanco import cleanco
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [247]:
official = pd.read_csv('sp500.csv')
sample = pd.read_csv('expanded_sample.csv')

In [258]:
sample[910:]

,Name,Variant
910,IHS Markit,HISD Markit
911,Ameriprise Financial,ameripris
912,Automatic Data Processing,auomatic data processing
913,L Brands,L BraRds
914,C.H. Robinson Worldwide,CH Robdnson
915,Incyte,Incyte
916,Alliant Energy,Alliant Enedgy
917,General Electric,General Electriac
918,ANSYS,adnsys
919,MasterCard,MasterCar


In [249]:
all_company =  official['Name']
correct_name = sample['Name']
var = sample['Variant']

In [250]:
def clean_co(series):
    for i in range(len(series)):
        x = cleanco(series[i])
        series[i] = x.clean_name()
        
    return series

correct_name = clean_co(correct_name)
var = clean_co(var)
all_company = clean_co(all_company)

In [251]:
def ngrams(string, n=2):
    string = re.sub(r'[,-./]|\sBD',r'', string)
    ngrams = zip(*[string[i:] for i in range(n)])
    return [''.join(ngram) for ngram in ngrams]

In [252]:
def predict(var, all_company):
    prediction = {}
    index = 0
    for i in var:
        copy = all_company.copy()
        copy.set_value(len(all_company),i)
        
        vectorizer = TfidfVectorizer(analyzer=ngrams)
        matrix = vectorizer.fit_transform(copy)

        similarities = cosine_similarity(matrix)
        prediction[index] = np.argmax([x for x in similarities[-1][:-1]])
        index += 1
    
    return prediction

def evaluate(pred, variant, true_value):
    correct = 0
    total = 0
    columns = ['variants', 'prediction', 'correct_name', "correctness"]
    df = pd.DataFrame(columns=columns)
    for i in range(len(pred)):
        total += 1
        predict = name[pred[i]]
        correctness = "N"
        if predict == true_value[i]:
            correct += 1
            correctness = "Y"
        
        df.loc[len(df)] = [variant[i], predict, true_value[i], correctness]

    print("Accuracy: ", correct * 100/ total, "%")
    
    return df
    

In [253]:
prediction = predict(var, all_company)
result = evaluate(prediction, var, correct_name)

Accuracy:  87.89682539682539 %


In [256]:
result[910:]

,variants,prediction,correct_name,correctness
910,HISD Markit,IHS Markit,IHS Markit,Y
911,ameripris,Ameriprise Financial,Ameriprise Financial,Y
912,auomatic data processing,Automatic Data Processing,Automatic Data Processing,Y
913,L BraRds,L Brands,L Brands,Y
914,CH Robdnson,C.H. Robinson Worldwide,C.H. Robinson Worldwide,Y
915,Incyte,Incyte,Incyte,Y
916,Alliant Enedgy,Alliant Energy,Alliant Energy,Y
917,General Electriac,General Electric,General Electric,Y
918,adnsys,Synopsys,ANSYS,N
919,MasterCar,MasterCard,MasterCard,Y
